# RO High Pressure Pump Power Maximization Problem

Reverse Osmosis (RO) is a very effective and important process for desalination and water waste reclamation. Assume that it is required to maximize the RO high pressure pump power. This power depends of a number of parameters according to the following equation:

$HP=\frac{M_d^2+1200 \times M_d \times N_v \times \Delta \pi}{3600 \times N_v \times RR \times \eta \times \rho}$

where, 

$HP$ is the RO high pressure pump power in kW

$M_d$ is the RO productivity in m3/d and in the range of $41.67< M_d <416.67$ m3/d

$N_v$ is number of pressure vessels and ranged as $1< N_v<200$

$\Delta \pi$ is the net osmotic pressure across the membrane and in the range $1400<\Delta \pi<2600$ kPa

$RR$ is the recovery ratio and ranged as $1< RR <50%$

$\eta$ the efficiency of the high pressure pump and in the range of $0.70< \eta <0.85$ and

$\rho$ is density of water

### Define the problem

In [7]:
import random

# Define the range for each decision variable
md_range = (41.67, 416.67)
nv_range = (1, 200)
delta_range = (1400, 2600)
rr_range = (1, 50)
eta_range = (0.70, 0.85)
rho = 1000  # density of water in kg/m^3

# Define the objective function
def ro_pump_power(X):
    md, nv, delta, rr, eta=X
    return (md ** 2 + 1200 * md * nv * delta) / (nv * rr * 3600 * eta * rho)

## Solving the problem using Ant Colony Optimization (ACO) algorithm

### Initialization

In [8]:
# Define the ACO parameters
num_ants = 100
num_iterations = 300
evaporation_rate = 0.7
pheromone_deposit = 1
initial_pheromone = 0.25

# Initialize the pheromone matrix
pheromone_matrix = [[initial_pheromone] * 5 for _ in range(num_ants)]

# Initialize the best solution and its corresponding power
best_solution = None
best_power = float('-inf')

### Start the ACO algorithm

In [9]:
for _ in range(num_iterations):
    solutions = []
    powers = []

    # Construct solutions for each ant
    for ant in range(num_ants):
        # Randomly select decision variables within their respective ranges
        md = random.uniform(md_range[0], md_range[1])
        nv = random.uniform(nv_range[0], nv_range[1])
        delta = random.uniform(delta_range[0], delta_range[1])
        rr = random.uniform(rr_range[0], rr_range[1])
        eta = random.uniform(eta_range[0], eta_range[1])

        # Calculate the power for the current solution
        soln=(md, nv, delta, rr, eta)
        power = ro_pump_power(soln)

        # Store the solution and its power
        solutions.append((md, nv, delta, rr, eta))
        powers.append(power)

        # Update the best solution if necessary
        if power > best_power:
            best_solution = (md, nv, delta, rr, eta)
            best_power = power

    # Update pheromone trails based on the power of each solution
    for ant in range(num_ants):
        for i in range(5):
            pheromone_matrix[ant][i] *= evaporation_rate
            if solutions[ant][i] == best_solution[i]:
                pheromone_matrix[ant][i] += pheromone_deposit / powers[ant]

### Print the best solution and its power

In [10]:
print("Optimal Solution:")
print("Md:", format(best_solution[0], '.2f'))
print("Nv:", format(best_solution[1], '.2f'))
print("Delta:", format(best_solution[2], '.2f'))
print("RR:", format(best_solution[3], '.2f'))
print("Eta:", format(best_solution[4], '.2f'))
print("Optimal HP:", format(best_power, '.2f'))


Optimal Solution:
Md: 404.10
Nv: 7.39
Delta: 2536.93
RR: 1.05
Eta: 0.76
Optimal HP: 425.75


## Solving the problem using scipy

In [11]:
from scipy.optimize import minimize

# Define the objective function with negative sign for minimization as per scipy's requirement
def ro_pump_power(X):
    md, nv, delta, rr, eta=X
    return -(md ** 2 + 1200 * md * nv * delta) / (nv * rr * 3600 * eta * rho) 

# Set the bounds for the decision variables
bounds = [md_range, nv_range, delta_range, rr_range, eta_range]
x0=[200, 100, 2000, 25, 0.75]  # initial guess

# Solve the optimization problem using Sequential Least Squares Programming (SLSQP)
result = minimize(ro_pump_power, x0, bounds=bounds, method='SLSQP')

### Print the best solution and its power

In [12]:
print("Optimal Solution:")
print("Md:", format(result.x[0], '.2f'))
print("Nv:", format(result.x[1], '.2f'))
print("Delta:", format(result.x[2], '.2f'))
print("RR:", format(result.x[3], '.2f'))
print("Eta:", format(result.x[4], '.2f'))
print("Optimal HP:", format(-result.fun, '.2f'))

Optimal Solution:
Md: 416.67
Nv: 99.98
Delta: 2600.00
RR: 1.00
Eta: 0.70
Optimal HP: 515.88
